In [ ]:
from warnings import warn
from IPython.display import HTML
def from_scratch():
    !pip install gensim >> ~/gensim.log
    !pip install spacy-nightly >> ~/spacy.log
    !python -m spacy download en >> ~/spacy.log
    !jupyter nbextension enable --py --sys-prefix widgetsnbextension
    warn("Refresh notebook to reflect widget availability.")

    
from_scratch()

In [2]:
from static_html import syntax
HTML(syntax)

In [3]:
from static_html import morphology
HTML(morphology)

,Effect,Morpheme
,,
un,Negation,Prefix
reach,Verb,Root
able,To Adjective,Suffix


In [4]:
from static_html import morph_and_syntax
HTML(morph_and_syntax)

In [5]:
from static_html import valency
HTML(valency)

In [15]:
spacy.load?

In [14]:
import spacy
nlp = spacy.load('en')


AttributeError: 'PosixPath' object has no attribute 'endswith'

In [13]:
doc = nlp(" i like baseball!")
from spacy import displacy
displacy.serve(doc, style='dep')

NameError: name 'nlp' is not defined

In [ ]:
!pip install 